In [1]:
import os
import json
import torch
import torchvision
from torchvision import transforms
from torchvision.datasets import CocoDetection
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt

# Paths
# TRAIN_PATH = '/kaggle/input/coco-2017-dataset/coco2017/train2017'
# VAL_PATH = '/kaggle/input/coco-2017-dataset/coco2017/val2017'
# ANNOTATIONS_PATH = '/kaggle/input/coco-2017-dataset/coco2017/annotations'
# WORKING_DIR = '/kaggle/working'

TRAIN_PATH = 'D:/Download/JDownloader/MSCOCO/images/train2017'
VAL_PATH = 'D:/Download/JDownloader/MSCOCO/images/val2017'
ANNOTATIONS_PATH = 'D:/Download/JDownloader/MSCOCO/annotations'
WORKING_DIR = 'D:/Projetos/Mestrado/2024_Topicos_Esp_Sist_Informacao/ARTIGO_FINAL/object_detection_model_compare/working'



# Categories of interest
CATEGORIES = {'person': 1, 'cat': 17, 'dog': 18}

def filter_annotations(annotations_path, categories):
    with open(os.path.join(annotations_path, 'instances_train2017.json')) as f:
        coco_data = json.load(f)

    images = []
    annotations = []
    category_ids = set(categories.values())

    for ann in coco_data['annotations']:
        if ann['category_id'] in category_ids:
            annotations.append(ann)

    image_ids = {ann['image_id'] for ann in annotations}
    for img in coco_data['images']:
        if img['id'] in image_ids:
            images.append(img)

    return images[:3000], annotations[:3000]

class FilteredCocoDataset(CocoDetection):
    def __init__(self, img_folder, ann_file, transform=None):
        super().__init__(img_folder, ann_file)
        self.transform = transform

    def __getitem__(self, idx):
        img, target = super().__getitem__(idx)
        if self.transform:
            img = self.transform(img)
        # Ensure targets are dictionaries with tensors
        target = [{k: torch.tensor(v) if isinstance(v, list) else v for k, v in t.items()} for t in target]
        return img, target

# Filter and prepare dataset
images, annotations = filter_annotations(ANNOTATIONS_PATH, CATEGORIES)

with open(os.path.join(WORKING_DIR, 'filtered_instances.json'), 'w') as f:
    json.dump({'images': images, 'annotations': annotations}, f)

dataset = FilteredCocoDataset(
    TRAIN_PATH,
    os.path.join(WORKING_DIR, 'filtered_instances.json'),
    transform=transforms.Compose([transforms.ToTensor()])
)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

def collate_fn(batch):
    images, targets = zip(*batch)
    return list(images), list(targets)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)

# Load pre-trained model and modify for COCO dataset
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
num_classes = len(CATEGORIES) + 1  # +1 for background
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)

# Training setup
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)
num_epochs = 10

def train_one_epoch(model, optimizer, data_loader, device, epoch):
    model.train()
    running_loss = 0
    for images, targets in data_loader:
        images = list(img.to(device) for img in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        optimizer.zero_grad()
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        losses.backward()
        optimizer.step()

        running_loss += losses.item()
    print(f"Epoch {epoch}, Loss: {running_loss/len(data_loader)}")
    return running_loss / len(data_loader)

def evaluate(model, data_loader, device):
    model.eval()
    with torch.no_grad():
        for images, targets in data_loader:
            images = list(img.to(device) for img in images)
            _ = model(images)

train_losses = []

for epoch in range(num_epochs):
    train_loss = train_one_epoch(model, optimizer, train_loader, device, epoch)
    train_losses.append(train_loss)
    evaluate(model, test_loader, device)

# Save the model
model_save_path = os.path.join(WORKING_DIR, 'fasterrcnn_coco.pth')
torch.save(model.state_dict(), model_save_path)

# Plot metrics
plt.figure(figsize=(10, 5))
plt.plot(range(num_epochs), train_losses, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.legend()
plt.savefig(os.path.join(WORKING_DIR, 'training_loss.png'))
plt.show()


loading annotations into memory...
Done (t=0.05s)
creating index...
index created!


C:\Users\lauro\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\lauro\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


AttributeError: 'list' object has no attribute 'items'